In [ ]:
#features when aggregated by mean. Hopefully this shows how you can incorporate more categorical data into your kernel.

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn import model_selection, preprocessing, metrics
import warnings
import datetime
warnings.filterwarnings("ignore")
import os
print(os.listdir("input"))
import sklearn
np.random.seed(1)
from bayes_opt import BayesianOptimization
pd.set_option('display.max_columns', 500)
import gc
from sklearn.model_selection import KFold


In [ ]:
#custom libraries
from memory_usage import *
from mean_encoding import *
from Preprocessing import *

In [ ]:
#Loading Train and Test Data
df_train = pd.read_csv("input/train_processed.csv",index_col=0)
df_test = pd.read_csv("input/test_processed.csv",index_col=0)

df_train = reduce_mem_usage(df_train)
df_test = reduce_mem_usage(df_test)

In [ ]:
#drop duplicated columns
df_test = drop_same_col(df_test)
df_train = drop_same_col(df_train)

In [ ]:
scaling_number = 33.21875

In [ ]:
#target scaling
def targ_descal(a,scaling_number):
    return a*scaling_number
    #return (np.exp(a)-2)*scaling_number
    #return a

In [ ]:
#function for getting embeddings from meta-data
def aggregate_num(df,df_new,num):
    
    
    #numerical features processing
    for i in num:
        #mean
        data = df_new.groupby('card_id')[i].mean()
        columns = df_new.groupby('card_id')[i].mean().index
        buf = pd.DataFrame({'card_id': columns.values, '{}_mean'.format(i): data.values})
        df = pd.merge(df,buf,on='card_id',how='left')
        
        #sum
        data = df_new.groupby('card_id')[i].sum()
        columns = df_new.groupby('card_id')[i].sum().index
        buf = pd.DataFrame({'card_id': columns.values, '{}_sum'.format(i): data.values})
        df = pd.merge(df,buf,on='card_id',how='left')

        #max
        data = df_new.groupby('card_id')[i].max()
        columns = df_new.groupby('card_id')[i].max().index
        buf = pd.DataFrame({'card_id': columns.values, '{}_max'.format(i): data.values})
        df = pd.merge(df,buf,on='card_id',how='left')
        
        #min
        data = df_new.groupby('card_id')[i].min()
        columns = df_new.groupby('card_id')[i].min().index
        buf = pd.DataFrame({'card_id': columns.values, '{}_min'.format(i): data.values})
        df = pd.merge(df,buf,on='card_id',how='left')
        
        #median
        data = df_new.groupby('card_id')[i].median()
        columns = df_new.groupby('card_id')[i].median().index
        buf = pd.DataFrame({'card_id': columns.values, '{}_median'.format(i): data.values})
        df = pd.merge(df,buf,on='card_id',how='left')
        
        #std
        data = df_new.groupby('card_id')[i].std()
        columns = df_new.groupby('card_id')[i].std().index
        buf = pd.DataFrame({'card_id': columns.values, '{}_std'.format(i): data.values})
        df = pd.merge(df,buf,on='card_id',how='left')
        

        
    del buf,data,columns
    gc.collect()
    return df

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
#main_cat = list(df_train.drop(['target','card_id','elapsed_time_main'],axis=1).columns)
#for i in main_cat:
    #print(i,df_train[i].unique().shape)

In [ ]:
#nan-features:

df_train['merchant_group_id_merchant_new_mf'][df_train['merchant_group_id_merchant_new_mf'] != 35] = 0
df_train['merchant_group_id_merchant_new_mf'][df_train['merchant_group_id_merchant_new_mf'] != 0] = 1
df_test['merchant_group_id_merchant_new_mf'][df_test['merchant_group_id_merchant_new_mf'] != 35] = 0
df_test['merchant_group_id_merchant_new_mf'][df_test['merchant_group_id_merchant_new_mf'] != 0] = 1


df_train['merchant_group_id_merchant_hist_mf'][df_train['merchant_group_id_merchant_hist_mf'] != 35] = 0
df_train['merchant_group_id_merchant_hist_mf'][df_train['merchant_group_id_merchant_hist_mf'] != 0] = 1
df_test['merchant_group_id_merchant_hist_mf'][df_test['merchant_group_id_merchant_hist_mf'] != 35] = 0
df_test['merchant_group_id_merchant_hist_mf'][df_test['merchant_group_id_merchant_hist_mf'] != 0] = 1

In [ ]:
test_na = df_test.columns[df_test.isnull().any()].tolist()
train_na = df_train.columns[df_train.isnull().any()].tolist()
na_cat = list(set(test_na).intersection(train_na))

In [ ]:
for i in na_cat:
    df_train['{}_n'.format(i)] = 0
    df_train['{}_n'.format(i)][df_train[i].isnull()] = 1

for i in na_cat:
    df_test['{}_n'.format(i)] = 0
    df_test['{}_n'.format(i)][df_test[i].isnull()] = 1
    
del na_cat,test_na,train_na
gc.collect()

In [ ]:
main_cat = list(df_train.drop(['target','card_id','elapsed_time_main'],axis=1).columns)

In [ ]:
df_hist_trans = pd.read_csv("input/historical_transactions.csv",index_col=0)
print('Loading of hist data is finished')

df_new_trans = pd.read_csv("input/new_merchant_transactions.csv",index_col=0)
print('Loading of new data is finished')

merchants = pd.read_csv("input/merchants.csv")
print('Loading of data is finished')

df_hist_trans = reduce_mem_usage(df_hist_trans)
df_new_trans = reduce_mem_usage(df_new_trans)
merchants = reduce_mem_usage(merchants)

In [ ]:
df_hist_trans = drop_same_col(df_hist_trans)
df_new_trans = drop_same_col(df_new_trans)
merchants = drop_same_col(merchants)
gc.collect()

In [ ]:
df_hist_trans = merged_data_processing(df_hist_trans)
df_new_trans = merged_data_processing(df_new_trans)
merchants = merchant_proc(merchants)

In [ ]:
df_hist_trans = pd.merge(df_hist_trans,merchants,on='merchant_id',how='left')
df_new_trans = pd.merge(df_new_trans,merchants,on='merchant_id',how='left')

df_new_trans['merchant_id_n'] = 0
df_new_trans['merchant_id_n'][df_new_trans['merchant_id'].isnull()] =1
df_new_trans = rename_col(df_new_trans,'new')
df_new_trans.head()

df_hist_trans['merchant_id_n'] = 0
df_hist_trans['merchant_id_n'][df_hist_trans['merchant_id'].isnull()] =1
df_hist_trans = rename_col(df_hist_trans,'hist')
df_hist_trans.head()

targ = df_train[['card_id','target']].copy()

df_new_trans = pd.merge(df_new_trans,targ,on='card_id',how='left')
df_hist_trans = pd.merge(df_hist_trans,targ,on='card_id',how='left')

del targ,merchants
gc.collect()

df_new_trans = reduce_mem_usage(df_new_trans)
df_hist_trans = reduce_mem_usage(df_hist_trans)

In [ ]:
df_new_trans['merchant_group_id_merchant_new'][df_new_trans['merchant_group_id_merchant_new'] != 35] = 0
df_new_trans['merchant_group_id_merchant_new'][df_new_trans['merchant_group_id_merchant_new'] != 0] = 1


df_hist_trans['merchant_group_id_merchant_hist'][df_hist_trans['merchant_group_id_merchant_hist'] != 35] = 0
df_hist_trans['merchant_group_id_merchant_hist'][df_hist_trans['merchant_group_id_merchant_hist'] != 0] = 1

In [ ]:
hist_num = ['purchase_amount_hist','numerical_1_merchant_hist','numerical_2_merchant_hist','avg_sales_lag3_merchant_hist',
          'avg_purchases_lag3_merchant_hist','avg_sales_lag6_merchant_hist','avg_purchases_lag6_merchant_hist',
          'avg_sales_lag12_merchant_hist','avg_purchases_lag12_merchant_hist']
buf = df_hist_trans.drop(['card_id','merchant_id','target'],axis=1)
hist_cat_enc = list(buf.drop(hist_num,axis=1).columns)
del buf
gc.collect()

new_num = ['purchase_amount_new','numerical_1_merchant_new','numerical_2_merchant_new','avg_sales_lag3_merchant_new',
          'avg_purchases_lag3_merchant_new','avg_sales_lag6_merchant_new','avg_purchases_lag6_merchant_new',
          'avg_sales_lag12_merchant_new','avg_purchases_lag12_merchant_new']
buf = df_new_trans.drop(['card_id','merchant_id','target'],axis=1)
new_cat_enc = list(buf.drop(new_num,axis=1).columns)
del buf
gc.collect()

In [ ]:
for i in new_cat_enc:
    print(i,df_new_trans[i].unique().shape)

In [ ]:
new_cat_enc,new_cat = divide_cat(df_new_trans,new_cat_enc)
hist_cat_enc,hist_cat = divide_cat(df_hist_trans,hist_cat_enc)

gc.collect()

In [ ]:
data = df_hist_trans['card_id'].value_counts()
columns = data.index
buf = pd.DataFrame({'card_id': columns.values, 'card_id_n_trans_hist': data.values})

df_train = pd.merge(df_train,buf,on='card_id',how='left')
df_test = pd.merge(df_test,buf,on='card_id',how='left')



data = df_new_trans['card_id'].value_counts()
columns = data.index
buf = pd.DataFrame({'card_id': columns.values, 'card_id_n_trans_new': data.values})

df_train = pd.merge(df_train,buf,on='card_id',how='left')
df_test = pd.merge(df_test,buf,on='card_id',how='left')

df_test['card_id_n_trans_new'] = df_test['card_id_n_trans_new'].fillna(0)
df_test['card_id_n_trans_hist'] = df_test['card_id_n_trans_hist'].fillna(0)

df_train['card_id_n_trans_new'] = df_train['card_id_n_trans_new'].fillna(0)
df_train['card_id_n_trans_hist'] = df_train['card_id_n_trans_hist'].fillna(0)

In [ ]:
alpha_new = create_alpha(new_cat_enc)
alpha_hist = create_alpha(hist_cat_enc)
alpha_main = create_alpha(main_cat)

In [ ]:
def aggregate_num(df,df_new,num):
    
    
    #numerical features processing
    for i in num:
        #print(i)
        
        #mean
        data = df_new.groupby('card_id')[i].mean()
        columns = data.index
        buf = pd.DataFrame({'card_id': columns.values, '{}_mean'.format(i): data.values})
        df = pd.merge(df,buf,on='card_id',how='left')
        df['{}_mean'.format(i)] = df['{}_mean'.format(i)].fillna(0)
        
        #sum
        data = df_new.groupby('card_id')[i].sum()
        columns = data.index
        buf = pd.DataFrame({'card_id': columns.values, '{}_sum'.format(i): data.values})
        df = pd.merge(df,buf,on='card_id',how='left')
        df['{}_sum'.format(i)] = df['{}_sum'.format(i)].fillna(0)

        #max
        data = df_new.groupby('card_id')[i].max()
        columns = data.index
        buf = pd.DataFrame({'card_id': columns.values, '{}_max'.format(i): data.values})
        df = pd.merge(df,buf,on='card_id',how='left')
        df['{}_max'.format(i)] = df['{}_max'.format(i)].fillna(0)
        
        #min
        data = df_new.groupby('card_id')[i].min()
        columns = data.index
        buf = pd.DataFrame({'card_id': columns.values, '{}_min'.format(i): data.values})
        df = pd.merge(df,buf,on='card_id',how='left')
        df['{}_min'.format(i)] = df['{}_min'.format(i)].fillna(0)
        
        #median
        data = df_new.groupby('card_id')[i].median()
        columns = data.index
        buf = pd.DataFrame({'card_id': columns.values, '{}_median'.format(i): data.values})
        df = pd.merge(df,buf,on='card_id',how='left')
        df['{}_median'.format(i)] = df['{}_median'.format(i)].fillna(0)
        
        #std
        data = df_new.groupby('card_id')[i].std()
        columns = data.index
        buf = pd.DataFrame({'card_id': columns.values, '{}_std'.format(i): data.values})
        df = pd.merge(df,buf,on='card_id',how='left')
        df['{}_std'.format(i)] = df['{}_std'.format(i)].fillna(0)

        
    del buf,data,columns
    gc.collect()
    return df

In [ ]:
new_num += new_cat_enc

In [ ]:
param = {'num_leaves': int(round(53.17295519924188)),
             'min_data_in_leaf': int(round(47.28877693542961)),
             'feature_fraction' : max(min(0.5153314260276387, 1), 0),
             'max_depth': int(round(9.958894298585333)),
             "bagging_freq": int(round(80.11669785745563)),
             "bagging_fraction": round(min((max(0.9807565080094875,0),1)),1),
             "lambda_l1": round(min((max(0.7805291762864555,0),1)),1),
             "lambda_l2": round(min((max(0.11827442586893322,0),1)),1),
             "min_split_gain":  0.0526629838532571,
             "min_child_weight": 8.024311083043273,
             'objective':'regression',
             "boosting": "gbdt",
             "verbosity": -1,
             "bagging_seed": 11,
             "metric": 'rmse',
             'learning_rate': 0.005,}

In [ ]:
target_colum = 'target'

In [ ]:
def score_calc(score,num_seeds,n_splits,n_boost):
    
    score = 1/score;

    novel = np.zeros(shape=(n_boost,num_seeds*n_splits))

    for r in range(n_boost):
        for t in range(num_seeds*n_splits):
            novel[r,t] = score[randint(0, t)]

    m = np.zeros(shape=(n_boost,1))

    for r in range(n_boost):
        m[r] = np.mean(novel[r,:])

    val_precision = (np.mean(m))
    val_recall = 1/(np.percentile(m,99.7) - np.percentile(m,0.3))

    return ((val_precision*val_recall)/(val_recall+val_precision))

In [ ]:
def fill_na(df,glob_mean):
    df = df.fillna(glob_mean)
    return df

In [ ]:
gc.collect()

In [ ]:
from sklearn.metrics import mean_squared_error
from random import randint

def lgb_eval(num_leaves, min_data_in_leaf, max_depth, feature_fraction, bagging_freq, bagging_fraction, lambda_l1, lambda_l2, min_split_gain, min_child_weight):
    param = {'num_leaves': int(round(num_leaves)),
             'min_data_in_leaf': int(round(min_data_in_leaf)),
             'feature_fraction' : max(min(feature_fraction, 1), 0),
             'max_depth': int(round(max_depth)),
             "bagging_freq": int(round(bagging_freq)),
             "bagging_fraction": round(min((max(bagging_fraction,0),1)),1),
             "lambda_l1": round(min((max(lambda_l1,0),1)),1),
             "lambda_l2": round(min((max(lambda_l2,0),1)),1),
             "min_split_gain": min_split_gain,
             "min_child_weight": min_child_weight,
             'objective':'regression',
             "boosting": "gbdt",
             "verbosity": -1,
             "bagging_seed": 11,
             "metric": 'rmse',
             'learning_rate': 0.005,
            }

    num_seeds = 1 #num seeds
    n_splits = 5 #n folds
    n_boost = 100000 #n of samples for bootstrapping

    droplist=['card_id','target']

    val_score = np.zeros(shape=(num_seeds*n_splits,1))
    for sed in range(num_seeds):
        np.random.seed(sed)
        folds = KFold(n_splits=n_splits, shuffle=True, random_state=sed)
        oof = np.zeros(len(df_train))
        predictions = np.zeros(len(df_test))

        for fold_, (trn_idx, val_idx) in enumerate(folds.split(df_train.values)):
            print('-')
            print("Fold {}".format(fold_ + 1))
            print('Seed {}'.format(sed + 1))

            train = df_train.iloc[trn_idx]
            target = df_train[target_colum].copy()
            val = df_train.iloc[val_idx]



            train_new = df_new_trans[df_new_trans['card_id'].isin(train['card_id'])]
            val_new = df_new_trans[df_new_trans['card_id'].isin(val['card_id'])]

            train_new = train_enc(train_new,new_cat_enc,target_colum,alpha_new)
            val_new,gb_new,dict_new = test_enc(train_new,val_new,new_cat_enc,target_colum,alpha_new)

            train = aggregate_num(train,train_new,new_num)
            val = aggregate_num(val,val_new,new_num)


            #applying mean target encoding
            train = train_enc(train,main_cat,target_colum,alpha_main)
            val,gb_main,dict_main = test_enc(train,val,main_cat,target_colum,alpha_main)



            #df_test_pred = pred_enc(df_test,dict_main,main_cat)

            train = fill_na(train,gb_main)
            val = fill_na(val,gb_main)

            val = val.drop(droplist,axis=1)
            train = train.drop(droplist,axis=1)

            trn_data = lgb.Dataset(train, label=target.iloc[trn_idx])
            val_data = lgb.Dataset(val, label=target.iloc[val_idx])

            

            num_round = 1000000
            
            clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=None, early_stopping_rounds=1000)
            
            oof[val_idx] = targ_descal(clf.predict(val, num_iteration=clf.best_iteration),scaling_number)
            val_score[fold_+sed*n_splits] = np.sqrt(mean_squared_error(targ_descal(target.iloc[val_idx],scaling_number), oof[val_idx]))
            #predictions += num_target_descaling(clf.predict(df_test_pred, num_iteration=clf.best_iteration),scaling_number) / n_splits*num_seeds
    
    
    return score_calc(val_score,num_seeds,n_splits,n_boost)

In [ ]:
#bayesian hyperopt
lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (20, 300),
                                        'min_data_in_leaf':(1,100),
                                        'max_depth': (1,18),
                                        'feature_fraction': (0.1, 1),
                                        'bagging_fraction': (0.1, 1),
                                        'bagging_freq': (1, 100),
                                        'lambda_l1': (5, 10),
                                        'lambda_l2': (5, 10),
                                        'min_split_gain': (0.001, 0.1),
                                        'min_child_weight': (1, 50)}, random_state=0)
init_round = 10
opt_round = 10
lgbBO.maximize(init_points=init_round, n_iter=opt_round)

In [ ]:
"""
убрать отображение фичей
сделать скейлинг
сделать сохранение параметров
уменьшить количество валидационных фолдов до 3
уменьшить количество фолдов для mean mean_encoding до 3
посмотреть как можно поменять loss в lightgbm
"""